In [14]:
import json
import os

import chromadb
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from chromadb.utils import embedding_functions
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer

In [15]:
# https://docs.trychroma.com/guides
CHROMA_DATA_PATH = "chroma_data/"
# EMBED_MODEL = "all-MiniLM-L6-v2"
EMBED_MODEL = "all-mpnet-base-v2"
COLLECTION_NAME = "arxiv_papers"
BATCH_SIZE = 5000

CHROMA_DATA_PATH = os.path.join(CHROMA_DATA_PATH, EMBED_MODEL)

In [16]:
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

parquet_path = '../data/arxiv_metadata_sample.parquet.gzip'
arxiv_df = pd.read_parquet(parquet_path)

print(arxiv_df.shape)

(119503, 14)


In [17]:
def text_processing(sample):
    title = sample['title']
    abstract = sample['abstract']

    # remove special characters
    title = title.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    abstract = abstract.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')

    # remove multiple spaces
    title = ' '.join(title.split())
    abstract = ' '.join(abstract.split())

    return f"Title: {title} - Abstract: {abstract}"

In [18]:
arxiv_df['text'] = arxiv_df.apply(text_processing, axis=1)
arxiv_df.head(3)

,id,title,abstract,categories,update_date,title_words,abstract_words,mapped_categories,amount_categories,update_year,super_categories,amount_super_categories,super_categories_str,super_category,text
0,2306.01499,Can LLMs like GPT-4 outperform traditional AI ...,Recent investigations show that large langua...,"[cs.CL, cs.LG]",2023-06-05,15,163,"[Computation and Language, Machine Learning]",2,2023,"[Artificial Intelligence, Artificial Intellige...",2,Computer Science and Artificial Intelligence,Artificial Intelligence,Title: Can LLMs like GPT-4 outperform traditio...
1,2312.05019,Vision-based Learning for Drones: A Survey,Drones as advanced cyber-physical systems ar...,"[cs.RO, cs.AI]",2024-01-03,6,181,"[Robotics, Artificial Intelligence]",2,2024,"[Artificial Intelligence, Artificial Intellige...",2,Computer Science and Artificial Intelligence,Artificial Intelligence,Title: Vision-based Learning for Drones: A Sur...
2,2307.14359,A new derivative-free optimization method: Gau...,Optimization methods are essential in solvin...,"[math.OC, cs.LG]",2023-07-28,8,114,"[Optimization and Control, Machine Learning]",2,2023,"[Engineering and Technology, Artificial Intell...",2,Computer Science and Artificial Intelligence E...,Artificial Intelligence,Title: A new derivative-free optimization meth...


In [19]:
def create_metadatas(arxiv_df):
    metadatas = []
    for _, row in arxiv_df.iterrows():
        metadatas.append({
            "update_date": row['update_date'],
            "title_words": row['title_words'],
            "abstract_words": row['abstract_words'],
            "super_category": row['super_category'],
            "mapped_categories": ";".join(row['mapped_categories']),
        })

    return metadatas

In [20]:
def create_collection(client, collection_name, embedding_function):
    collection = client.create_collection(
        name=collection_name,
        embedding_function=embedding_function,
        metadata={"hnsw:space": "cosine"},
        get_or_create=True,
    )

    return collection

def delete_collection_data(client, collection, collection_name):
    print(f"Deleting data from collection {collection_name} with {collection.count()} documents")
    client.delete_collection(collection_name)

def get_random_samples_from_collection(collection, n_samples):
    collection_ids = collection.get()["ids"]
    random_ids = np.random.choice(collection_ids, n_samples, replace=False).tolist()
    documents = collection.get(ids=random_ids)
    return documents

def upsert_data(collection, arxiv_df, metadatas, batch_size):
    for i in tqdm(range(0, len(arxiv_df), batch_size)):
        collection.upsert(
            documents=arxiv_df['text'].iloc[i:i + batch_size].tolist(),
            ids=arxiv_df['id'].iloc[i:i + batch_size].tolist(),
            metadatas=metadatas[i:i + batch_size],
        )

In [21]:
# delete the collection if it exists
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL,
    device="cuda",
)

collection = create_collection(client, COLLECTION_NAME, embedding_func)

# delete if you want to start fresh but then you need to create the collection again
# delete_collection_data(client, collection, COLLECTION_NAME)
# collection = create_collection(client, COLLECTION_NAME, embedding_func)

# create metadatas
metadatas = create_metadatas(arxiv_df)

# upsert data (insert or update if exists)
# upsert_data(collection, arxiv_df, metadatas, BATCH_SIZE)

w:\Workspaces\Python\Studium\Master\ArxivAbstractProject\.venv311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Deleting data from collection arxiv_papers with 0 documents


  0%|          | 0/24 [00:00<?, ?it/s]

In [23]:
# Define query
words_per_line = 10
# define papers to show
top_n_papers = 3
query = "Deep Learning and veganism"
print("Query:\n", query, "\n")
query_results = collection.query(query_texts=[query], n_results=top_n_papers)
for _id, _doc, _dist, _meta in zip(query_results["ids"][0], query_results["documents"][0], query_results["distances"][0], query_results["metadatas"][0]):
    print(f"#####   ID: {_id}   #####")
    print(f"Distance: {_dist}")
    print(f"Metadata: {_meta}")
    _doc_lines = _doc.split()
    for i in range(0, len(_doc_lines), words_per_line):
        print(" ".join(_doc_lines[i:i + words_per_line]))
    print("\n")

Query:
 Deep Learning and veganism 

#####   ID: 2012.05995   #####
Distance: 0.49146878719329834
Metadata: {'abstract_words': 165, 'mapped_categories': 'Genomics', 'super_category': 'Biological Sciences and Medicine', 'title_words': 10, 'update_date': '2022-03-15'}
Title: Towards a robust out-of-the-box neural network model for genomic
data - Abstract: The accurate prediction of biological features from
genomic data is paramount for precision medicine and sustainable agriculture.
For decades, neural network models have been widely popular in
fields like computer vision, astrophysics and targeted marketing given their
prediction accuracy and their robust performance under big data settings.
Yet neural network models have not made a successful transition
into the medical and biological world due to the ubiquitous
characteristics of biological data such as modest sample sizes, sparsity,
and extreme heterogeneity. Here, we investigate the robustness, generalization potential
and predictio

In [24]:
response = collection.get(include=["metadatas", "documents", "embeddings"])
df = pd.DataFrame({
    "id": response["ids"],
    "document": response["documents"],
    "embedding": response["embeddings"],
    "super_category": [x["super_category"] for x in response["metadatas"]],
})
# all_categories = df["categories"].explode().unique()
# cat_mapping = {cat: i for i, cat in enumerate(all_categories)}
# df["cat_id"] = df["categories"].apply(lambda x: cat_mapping[x])
# df.loc[:, all_categories] = df["categories"].apply(lambda x: [1 if cat in x else 0 for cat in all_categories]).tolist()
df.head(3)

,id,document,embedding,super_category
0,1001.0039,"Title: TGCat, The Chandra Transmission Grating...","[0.05119134858250618, -0.018094409257173538, 0...",Astrophysics and Cosmology
1,1001.0051,Title: Lukewarm dark matter: Bose condensation...,"[0.018889665603637695, -0.08081771433353424, 0...",Astrophysics and Cosmology
2,1001.0057,Title: Arrested phase separation in reproducin...,"[-0.011269346810877323, -0.04270167648792267, ...",Biological Sciences and Medicine


In [ ]:
# from renumics import spotlight

# spotlight.show(df)

# Simple Classification Head

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


class EmbeddingDataset(Dataset):
    def __init__(self, df, cat_ids):
        self.ids = df["id"].values
        self.embeddings = torch.tensor(df["embedding"].tolist(), dtype=torch.float32)
        self.categories = torch.tensor(df["cat_id"].values, dtype=torch.long)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        return self.ids[idx], self.embeddings[idx], self.categories[idx]

class EmbeddingModel(torch.nn.Module):
    def __init__(self, n_emb_size, n_categories):
        super(EmbeddingModel, self).__init__()
        self.fc1 = torch.nn.Linear(n_emb_size, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.fc3 = torch.nn.Linear(256, n_categories)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# train val split
emb_dataset = EmbeddingDataset(df, cat_mapping)
train_size = int(0.8 * len(emb_dataset))
val_size = len(emb_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(emb_dataset, [train_size, val_size])

print(f"Train size: {len(train_dataset)}")
print(f"Val size: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmbeddingModel(len(df["embedding"].values[0]), len(all_categories)).to(device)

In [ ]:
def train(model, train_loader, val_loader, epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        for _, emb, cat in train_loader:
            emb, cat = emb.to(device), cat.to(device)
            optimizer.zero_grad()
            output = model(emb)
            loss = criterion(output, cat)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for _, emb, cat in val_loader:
                emb, cat = emb.to(device), cat.to(device)
                output = model(emb)
                _, predicted = torch.max(output.data, 1)
                total += cat.size(0)
                correct += (predicted == cat).sum().item()

            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Accuracy: {100 * correct / total}")

train(model, train_loader, val_loader, epochs=10)

# cluster

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

classes = arxiv_df['mapped_categories'].explode().unique()
classes = {c: i for i, c in enumerate(classes)}
num_classes = len(classes)
print(f"Number of classes: {num_classes}")
arxiv_df['class'] = arxiv_df['mapped_categories'].apply(lambda x: classes[x[0]])

In [ ]:
kmeans = KMeans(init="k-means++", n_clusters=num_classes, n_init=4, random_state=0)
estimator = make_pipeline(StandardScaler(), kmeans).fit(df["embedding"].tolist())
# estimator = make_pipeline(kmeans).fit(df["embedding"].tolist())

df["cluster"] = estimator.predict(df["embedding"].tolist())
merged_df = pd.merge(df[['id', 'cluster']], arxiv_df, on="id")

accuracy = metrics.accuracy_score(merged_df['class'], merged_df['cluster'])
print(f"Accuracy: {accuracy}")